In [ ]:
# default_exp apigateway

# apigateway schema for lambda proxy integration


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from dataclasses import field
from dataclasses import dataclass, field
from dataclasses_json import dataclass_json, Undefined
from pprint import pformat
from typing import Optional, List, Callable, Any
import ujson as json

@dataclass_json
@dataclass
class Response:
  '''
    parse response from apigateway
  '''
  body: str
  statusCode: int = 200
  headers: dict = field(default_factory = dict)
  @classmethod
  def parseBody(cls, dictInput:dict):
    response = cls.fromDict(dictInput)
    return response.body
  
  @classmethod
  def fromDict(cls, dictInput:dict):
    '''
      output object from Dict,
      dictInput should follow apigateway proxy integration
    '''
    body = dictInput.pop('body')
    return cls(
      body = json.loads(body),
      **dictInput
    )
  @classmethod
  def getReturn(cls, body:dict, headers:dict = {}, statusCode:int = 200)->dict:
    '''
      output dictionary which is suitable for apigateway proxy integration return
    '''
    returnObj = cls(
      body = json.dumps(body),
      headers = headers,
      statusCode = statusCode
                   ).to_dict()
    return returnObj
  @classmethod
  def returnError(cls, message:str, statusCode:int = 400, **kwargs)->dict:
    return cls.getReturn(statusCode = 400, body = {'error': message})
  @classmethod
  def returnSuccess(cls, body:dict = {}, **kwargs)->dict:
    return cls.getReturn(statusCode = 200, body = body, **kwargs)
  
@dataclass_json(undefined=Undefined.EXCLUDE)
@dataclass
class Event:
  '''
    parse event from apigateway
  '''
  body: str
  headers: dict = field(default_factory = dict)
  statusCode: int = 200
  def getBody(self,*args):
    try:
      return json.loads(self.body)
    except:
      return Event.parseBody(self,*args)
  def getProducts(self):
    return Products.from_json(self.body)
  def getKey(self, key='product'):
    return body.get(key)
  key = lambda self: json.loads(self.body)['key']
  firstKey = lambda self: next(iter(json.loads(self.body).items()))
  @classmethod
  def parseBody(cls, event, *args):
    return cls.from_dict(event).getBody()
  @classmethod
  def getInput(cls, body={},headers={},statusCode=200):
    return cls(body=json.dumps(body),headers=headers,statusCode=statusCode).to_dict()
  @classmethod
  def parseDataClass(cls, customClass, event):
    body = cls.getBody(event)
    try:
      return customClass.from_dict(body)
    except Exception as e:
      raise Exception(f'unable to parse input{e}, should have the schema {customClass.__doc__},\
        but the current input is {body}')
  
@dataclass_json
@dataclass
class Product:
  cprcode: str
  iprcode: str
  oprcode: str
  ordertype: str
  pr_abb: str
@dataclass_json
@dataclass
class Products:
  products: List[Product]

# Examples

In [ ]:
event = Event(body = json.dumps({'test':'test'})).to_dict()
print(event)

{'body': '{"test":"test"}', 'headers': {}, 'statusCode': 200}


In [ ]:
Event.parseBody(event)

{'test': 'test'}

In [ ]:
Response.returnSuccess(body = {'success':'true'})

{'body': '{"success":"true"}', 'statusCode': 200, 'headers': {}}

In [ ]:
Response.returnError(message='error')

{'body': '{"error":"error"}', 'statusCode': 400, 'headers': {}}

responseDict = Response.returnSuccess(body = {'success':'true'})
Response.fromDict(responseDict)

In [ ]:
Event.getInput(body={'input':'123'})

{'body': '{"input":"123"}', 'headers': {}, 'statusCode': 200}

## Parsing data class directly

In [ ]:
productDict = Product(
  cprcode='123',
  iprcode='123',
  oprcode= '12343',
  ordertype='3225',
  pr_abb='4563'
).to_dict()
event = Event.getInput(productDict)
Event.parseDataClass(Product, event)

Product(cprcode='123', iprcode='123', oprcode='12343', ordertype='3225', pr_abb='4563')